In [388]:
from numpy import hstack, ones, array, mat, tile, reshape, squeeze, eye, asmatrix
from numpy.linalg import inv
from pandas import read_csv, Series 
from scipy.linalg import kron
from scipy.optimize import fmin_bfgs
from scipy import stats, signal
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM, IV2SLS

In [389]:
# read in data
data = read_csv('EA3_data.csv')
# # filter to only results from paper
# data = data[data['Date'] < '1964']

In [390]:
# define portfolio returns vector
R = np.array(data.iloc[:,5:].values+1)
T, K = R.shape

# define V vector, vector of ones, Ndr, Ncf entries
V = np.array([np.ones((T)), data['NdrMinus'].values, data['Ncf']]).T

# define D = (1/T)R'V
D = (1/T)*(R.T @ V)

# define g(B) = DBeta-1
# since D is linear we can sovle analytically for B1gmm
# FN b1gmm = inv(D'WD)D'Wi for some weight matrix W
W = np.eye(10)
A = D.T @ W
DWD = inv(D.T @ W @ D)
beta_gmm1 = inv(A @ D) @ D.T @ W @ np.ones(10)
a,b,c = beta_gmm1

print(f"Estimate for a: {a}")
print(f"Estimate for b: {b}")
print(f"Estimate for c: {c}")

Estimate for a: 0.9920903001072756
Estimate for b: 0.42170025554557355
Estimate for c: -11.294426841715904


In [391]:
# now use the CLT to get an estimate for S using the variance of B
g = np.zeros((10,10))
for t in range(10):
    vb = V[t] @ beta_gmm1
    uhat = R[t]*vb - np.ones(10)
    g[t] = uhat

means = np.outer(np.ones(10), np.mean(g, axis=0))
demeaned = g - means
S = np.cov(demeaned, bias=True)

# newey west correction, note python doesn't have a fn like matlab's xcorr
# xcorr = signal.correlate2d(demeaned, demeaned)

var_beta_gmm1 = inv(T * A @ D) @ (A @ S @ A.T) @ inv(A @ D)
std_devs = np.sqrt(np.diag(var_beta_gmm1))
std_devs

array([4.11997966e-03, 1.28452307e+00, 8.02757927e+00])

In [393]:
# wald test
restr = np.array([1,1,0])
rtb = restr @ beta_gmm1
wald = rtb.T * (restr @ var_beta_gmm1 @ restr.T)**(-1) * rtb
pval = 1 - stats.chi2.cdf(wald, 1)

print(f"Wald test returns a value of {wald:.2f} with {pval:.3f} significance for a chi-squared distribution with 1 degrees of freedom.")

Wald test returns a value of 1.21 with 0.271 significance for a chi-squared distribution with 1 degrees of freedom.


In [394]:
# gamma estimate
gamma = -c/a

d_gamma = np.array([-c/(a**2),0,-1/a])
std_err = d_gamma.T @ var_beta_gmm1 @ d_gamma
std_err


65.56069116547305

In [395]:
# returning to the initial model, we'll use the inverse of S as the new weight matrix
W2 = inv(S)
A = D.T @ W2
beta_gmm2 = inv(A @ D) @ D.T @ W2 @ np.ones(10)
a,b,c = beta_gmm2

g = np.zeros((10,10))
for t in range(10):
    vb = V[t] @ beta_gmm2
    uhat = R[t]*vb - np.ones(10)
    g[t] = uhat

means = np.outer(np.ones(10), np.mean(g, axis=0))
demeaned = g - means
S = np.cov(demeaned, bias=True)

var_beta_gmm2 = inv(T * A @ D) @ (A @ S @ A.T) @ inv(A @ D)
std_devs = np.sqrt(np.diag(var_beta_gmm2))
std_a, std_b, std_c = std_devs

print(f"Estimate for a: {a}\n Std Err for a: {std_a}")
print(f"Estimate for b: {b}\n Std Err for b: {std_b}")
print(f"Estimate for c: {c}\n Std Err for c: {std_c}")

Estimate for a: 0.0361328125
 Std Err for a: 0.050931268790645697
Estimate for b: -20.125
 Std Err for b: 15.620499351813308
Estimate for c: -15.25
 Std Err for c: 66.81317235396027


In [396]:
# second stage wald test
restr = np.array([[1,1,0]]).T
beta = np.array([beta_gmm2]).T

rtb = restr.T @ beta
[[wald]] = rtb.T * (1/(restr.T @ var_beta_gmm2 @ restr)) * rtb
pval = 1 - stats.chi2.cdf(wald, 1)

print(f"Wald test returns a value of {wald:.2f} with {pval:.3f} significance for a chi-squared distribution with 1 degrees of freedom.")


Wald test returns a value of 1.67 with 0.197 significance for a chi-squared distribution with 1 degrees of freedom.


In [397]:
# second stage standard error
gamma = -c/a
d_gamma = np.array([[-c/(a**2)],[0],[-1/a]]).T
[[std_err]] = d_gamma @ var_beta_gmm2 @ d_gamma.T
std_err

1159150.9814236881

In [398]:
# test for overidentification
gb = np.array([D @ beta_gmm2 - np.ones(10)])

[[J]] = T * (gb @ inv(S) @ gb.T)
pval = 1 - stats.chi2.cdf(J, 7)

print(f"J test returns a value of {J:.2f} with {pval:.3f} significance for a chi-squared distribution with 7 degrees of freedom.")

J test returns a value of 310449888898177564672.00 with 0.000 significance for a chi-squared distribution with 7 degrees of freedom.
